<a href="https://colab.research.google.com/github/debaghtk/finetuning-poc/blob/main/finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/github/huggingface/notebooks/blob/main/peft_docs/en/tensorflow/clm-prompt-tuning.ipynb#scrollTo=buySkzBqU1zS

In [ ]:
!pip install transformers datasets

download model and dataset

In [14]:
from transformers import AutoModel as AutoModel, AutoTokenizer
from datasets import load_dataset

model_name = "bigscience/bloomz-560m"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

dataset = load_dataset("ought/raft", "twitter_complaints")

doing inference like TDD

In [ ]:
inputs = tokenizer(
    "@nationalgridus I have no water and the bill is current and paid. Can you do something about this?",
    return_tensors="pt",
)

output = model(**inputs)
print(type(output.logits))